In [1]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Extract CSVs into DataFrames
game = "game.csv"
game_df = pd.read_csv(game)

game_file_df

In [3]:
game_df

,game_id,season,type,date_time,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz
0,2012030221,20122013,P,2013-05-16T23:30:00Z,3,6,2,3,home win OT,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT
1,2012030222,20122013,P,2013-05-19T19:00:00Z,3,6,2,5,home win REG,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT
2,2012030223,20122013,P,2013-05-21T23:30:00Z,6,3,2,1,away win REG,right,Madison Square Garden,/api/v1/venues/null,America/New_York,-4,EDT
3,2012030224,20122013,P,2013-05-23T23:00:00Z,6,3,3,4,home win OT,right,Madison Square Garden,/api/v1/venues/null,America/New_York,-4,EDT
4,2012030225,20122013,P,2013-05-25T21:30:00Z,3,6,1,3,home win REG,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT
5,2012030311,20122013,P,2013-06-02T00:00:00Z,6,5,3,0,away win REG,left,CONSOL Energy Center,/api/v1/venues/null,America/New_York,-4,EDT
6,2012030312,20122013,P,2013-06-04T00:00:00Z,6,5,6,1,away win REG,left,CONSOL Energy Center,/api/v1/venues/null,America/New_York,-4,EDT
7,2012030313,20122013,P,2013-06-06T00:00:00Z,5,6,1,2,home win OT,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT
8,2012030314,20122013,P,2013-06-08T00:00:00Z,5,6,0,1,home win REG,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT
9,2012030231,20122013,P,2013-05-16T00:00:00Z,17,16,1,4,home win REG,right,United Center,/api/v1/venues/null,America/Chicago,-5,CDT


In [4]:
team_info = "team_info.csv"
team_info_df = pd.read_csv(team_info)

In [5]:
team_info_df

,team_id,franchiseId,shortName,teamName,abbreviation,link
0,1,23,New Jersey,Devils,NJD,/api/v1/teams/1
1,4,16,Philadelphia,Flyers,PHI,/api/v1/teams/4
2,26,14,Los Angeles,Kings,LAK,/api/v1/teams/26
3,14,31,Tampa Bay,Lightning,TBL,/api/v1/teams/14
4,6,6,Boston,Bruins,BOS,/api/v1/teams/6
5,3,10,NY Rangers,Rangers,NYR,/api/v1/teams/3
6,5,17,Pittsburgh,Penguins,PIT,/api/v1/teams/5
7,17,12,Detroit,Red Wings,DET,/api/v1/teams/17
8,28,29,San Jose,Sharks,SJS,/api/v1/teams/28
9,18,34,Nashville,Predators,NSH,/api/v1/teams/18


In [10]:
# Transform Game DataFrame

In [11]:
# Transform Team_Info DataFrame

In [ ]:
# Create database connection

In [ ]:
# Confirm tables

In [ ]:
# Load DataFrames into database